## Lectures
 - (Value functions approximation - RL by David Silver)[https://www.youtube.com/watch?v=UoPei5o4fps&list=PLqYmG7hTraZDM-OYHWgPebj2MfCFzFObQ&index=6]
    - Differentiable function approximators
    - Incremental methods
    - Batch methods (DQN)
 - (Advanced Q-learning algorithms - DRL UC Berkley by Sergey Levine)[https://www.youtube.com/watch?v=nZXC5OdDfs4&list=PLkFD6_40KJIznC9CDbVTjAF2oyt8_VAe3&index=7]
    - Replay Buffer
    - Double Q-learning
    - Continous actions (NAF,DDPG)
    - Pratical tips